Now there are only two methods to implement the entire logic.Exceptions like 2004 > season > 2019 and 1 > model code > 5 are also handled......Simply calling the method prediction_4_season will give the desired matrix for valid inputs and appropriate message for invalid input.

In [1]:
!pip install joblib

In [2]:

%matplotlib inline
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
from numpy import loadtxt
import os
import scipy
import matplotlib.pyplot as plt
from matplotlib import rcParams
import warnings
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
import seaborn as sb

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import LogisticRegressionCV
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
import xgboost as xgb

from xgboost import XGBClassifier
#from sklearn.model_selection import train_test_split
#from sklearn import joblib
import joblib
from joblib import dump, load
import pickle
from sklearn.preprocessing import LabelEncoder

In [3]:
#To view all columns in DF ,expanded without ellipses
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [4]:
def get_matRx_4_gvn_Year(year, df_4m_csv):
    groupby_Season = df_4m_csv.groupby('season')
        
    df_for_Year_Passed_1 = groupby_Season.get_group(year-1)
    groupby_Team_Avg_Values_1 = df_for_Year_Passed_1.groupby('team', as_index=False).mean()
    groupby_Team_Avg_Values_1 = groupby_Team_Avg_Values_1[['season', 'week','team', 'home', 'pass_attempts', 'pass_completions', 'pass_yards', 'net_pass_yards', 'pass_tds', 'rush_attempts', 'rush_yards', 'rush_tds', 'total_yards', 'first_downs', 'sacks', 'sacks_yards', 'pass_interceptions', 'fumbles', 'fumbles_lost', 'turnovers', 'time_of_possession', 'pentalties', 'penalty_yards', 'third_down_attempts', 'third_down_conversions', 'fourth_down_attempts', 'fourth_down_conversions','Score']]
        
    df_for_Year_Passed_2 = groupby_Season.get_group(year-2)
    groupby_Team_Avg_Values_2 = df_for_Year_Passed_2.groupby('team', as_index=False).mean()
    groupby_Team_Avg_Values_2 = groupby_Team_Avg_Values_2[['season', 'week','team', 'home', 'pass_attempts', 'pass_completions', 'pass_yards', 'net_pass_yards', 'pass_tds', 'rush_attempts', 'rush_yards', 'rush_tds', 'total_yards', 'first_downs', 'sacks', 'sacks_yards', 'pass_interceptions', 'fumbles', 'fumbles_lost', 'turnovers', 'time_of_possession', 'pentalties', 'penalty_yards', 'third_down_attempts', 'third_down_conversions', 'fourth_down_attempts', 'fourth_down_conversions','Score']]
    
    result = pd.concat([groupby_Team_Avg_Values_1, groupby_Team_Avg_Values_2], axis=0).dropna(axis=1).groupby('team', as_index=False).mean()
    result = result[['season', 'week','team', 'home', 'pass_attempts', 'pass_completions', 'pass_yards', 'net_pass_yards', 'pass_tds', 'rush_attempts', 'rush_yards', 'rush_tds', 'total_yards', 'first_downs', 'sacks', 'sacks_yards', 'pass_interceptions', 'fumbles', 'fumbles_lost', 'turnovers', 'time_of_possession', 'pentalties', 'penalty_yards', 'third_down_attempts', 'third_down_conversions', 'fourth_down_attempts', 'fourth_down_conversions','Score']]
    scaled = preprocessing.MinMaxScaler()
    scaled_result = scaled.fit_transform(result)
    return scaled_result

In [5]:
def prediction_4_season(season,model):
    if int(season) in range(2004, 2020) and int(model) in range(1, 6):
        
        win_loss_prdctd_data = pd.DataFrame(columns=['Team','0', '1', '2', '3','4', '5', '6','7', '8', '9','10', '11', '12', '13','14', '15', '16','17', '18', '19','20', '21', '22', '23','24', '25', '26','27', '28', '29', '30', '31'])
        win_loss_prdctd_data["Team"] = ['0', '1', '2', '3','4', '5', '6','7', '8', '9','10', '11', '12', '13','14', '15', '16','17', '18', '19','20', '21', '22', '23','24', '25', '26','27', '28', '29', '30', '31']
        switcher = { 
             1: './LR_model.sav', 2: './GB_model.sav', 3: './GB_best_model.sav', 4: './svm_model.sav', 5: './xgb_model.sav'
        }
        model = switcher.get(model, "nothing")
    
#   Load the data from the csv
        df=pd.read_csv("NFL_datasetForML.csv")
#   Create a list named team_Names and sort them alphabetically.....It will be used for relabelling the column and row label
        team_Names = df['team'].unique()
        team_Names = team_Names.tolist()
        team_Names = sorted(team_Names)
#   Drop unnecessary columns from the daataframe
        df_effective=df.drop(['Unnamed: 0','boxscore_id', 'roof', 'surface', 'temp', 'humidity', 'wind_chill', 'wind_mph', 'Won'], axis=1)
#   Label encoding of two columns having categorical data
        le = LabelEncoder()
        df_effective['team'] = le.fit_transform(df_effective['team'])
        df_effective['time_of_possession'] = le.fit_transform(df_effective['time_of_possession'])

        print("\033[1m" + "Prediction table for the Season/Year %i using the `%s` !" % (season, model.split('/')[1].split('.')[0].replace('_',' ').upper()))
#   Loading the desired model
        logit_model=joblib.load(model)
#   getting the result of prediction by passing the season specific data(by calling the method get_matRx_4_gvn_Year) to the 
#   model prepared
        LR_result=logit_model.predict_proba(get_matRx_4_gvn_Year(season,df_effective))
    
        for i, row in win_loss_prdctd_data.iterrows():
            for j in range(1,33):
                if(row['Team']!=str(j-1)):
                    x_Prob = LR_result[i]
                    y_Prob = LR_result[j-1]
#                     prob_diff = abs(x_Prob[1] - y_Prob[1])
                    row[j] = 1 if (x_Prob[1] > y_Prob[1]) else 0
                else:
                    row[j] = "Same Team"
                
#   Relabel the rows
        win_loss_prdctd_data["Team"] = team_Names
#   Add a new element Team in the list 
        team_Names.insert(0,'Team')
#   Relabel the columns headers with the new list 
        win_loss_prdctd_data.columns = team_Names
        win_loss_prdctd_data.set_index('Team')
        return win_loss_prdctd_data
    else:
        message = print("\033[1m" + "Sorry ! The Matrix can be generated only for 2004 <= season <= 2019 and 1 <= model <= 5.\n Please Enter relevant values..")
        return message

In [6]:
# Call the method prediction_4_season by passing the value for the year and model code according to which the probability 
# matrix is required.Here model is passed as codes as 1: LR Model, 2: GB Model, 3: GB_best Model, 4: svm Model, 5: xgb Model
# Note: When the below method will be called from front end....Year can be passed directly from the dropbox whereas for MODEL
# it is suggested to represent the user friendly name of the model as the TEXT property of the dropbox control and numerical
# codes as the VALUE property of the same..... 
prediction_Result = prediction_4_season(2012, 1)
prediction_Result

Prediction table for the Season/Year 2012 using the `LR MODEL` !


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator StandardScaler from version 0.20.3 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.3 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator Pipeline from version 0.20.3 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,Team,atl,buf,car,chi,cin,cle,clt,crd,dal,den,det,gnb,htx,jax,kan,mia,min,nor,nwe,nyg,nyj,oti,phi,pit,rai,ram,rav,sdg,sea,sfo,tam,was
0,atl,Same Team,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,0,1,1
1,buf,0,Same Team,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,car,0,1,Same Team,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
3,chi,0,1,1,Same Team,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,1,0,1,1,0,1,1,0,1,1,0,1,1
4,cin,0,1,1,0,Same Team,1,1,1,0,1,1,0,0,1,0,1,1,1,0,0,0,1,1,0,0,1,0,0,1,0,1,1
5,cle,0,1,1,0,0,Same Team,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1
6,clt,0,1,1,0,0,1,Same Team,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1
7,crd,0,0,0,0,0,0,0,Same Team,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,dal,0,1,1,0,1,1,1,1,Same Team,1,1,0,0,1,0,1,1,1,0,1,0,1,1,0,0,1,0,0,1,0,1,1
9,den,0,1,1,0,0,1,1,1,0,Same Team,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1


In [7]:
# Try calling the method by passing non permitted values and see the result
prediction_Result1 = prediction_4_season(2002, 1)
prediction_Result1

Sorry ! The Matrix can be generated only for 2004 <= season <= 2019 and 1 <= model <= 5.
 Please Enter relevant values..
